<a href="https://colab.research.google.com/github/ViviennePetracca/MST-Data-Analysis/blob/main/Research_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#From Spreadsheet ->  #2

#Have prepared: A spreadsheet with each dataset you want to stitch together
#A completely blank sheet labeled 'Master'
#If you need help converting from a CSV file to google sheets:
#Open a spreadsheet and click File -> Import. Then choose a CSV to upload.
# Column names must be in row 1 only

#Future: Upload CSV files directly into Colab and then make it into a spreadsheet

In [ ]:
# Import via Google Spreadsheets


from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

import pandas as pd
import numpy as np


wb = gc.open('Master Bird Spreadsheet')


In [ ]:
# Stores the sheets as ListOfFrames and combines them (Sheet1 will be labeled as ListOfFrames[0])

ListOfFrames = []

for ws in wb.worksheets():
  if ws.title != 'Master':
    rows = ws.get_all_values()
    df = pd.DataFrame.from_records(rows[1:],columns=rows[0])
    ListOfFrames.append(df)

    df = pd.concat(ListOfFrames)

In [ ]:
#Combining Columns if they don't have the exact same name:

df['Combined Number of Individuals'] = df['No of individuals'].fillna(df['Number of Individuals'])
Masterdf = df.drop(['No of individuals','Number of Individuals'],axis=1)


In [ ]:
#Delete rows that include x (Null values/Above or below a certain value/ a certain name)

#Null Values
#df = Masterdf.dropna()


#Column name, value 'if not number'
df = Masterdf[df.Species != 'Lesser Whitethroat']

df

In [ ]:
#Sort by Column

Masterdf.sort_values(by=['Date\n(mm-dd-yyyy)'])


In [ ]:
#Imports cleaned dataset to new Google Sheets

Masterlist = [Masterdf.columns.tolist()] + Masterdf.to_numpy().tolist()
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.create('Master Bird Data')

# Open our new sheet and add some data.
worksheet = gc.open('Master Bird Data').sheet1

MasterExport = Masterdf
worksheet.update_cells(MasterExport)
# Go to https://sheets.google.com to see your new spreadsheet.

In [ ]:
# By this point the dataset is cleaned: The next step is data analysis

In [ ]:
#Makes values in columns into floats (which is what the code uses for running data analysis tests)

Masterdf['Transect1'] = pd.to_numeric(Masterdf['Transect'],errors = 'coerce')
Masterdf['Combined Number of Individuals1'] = pd.to_numeric(Masterdf['Combined Number of Individuals'],errors = 'coerce')

In [ ]:
print(df.dtypes)

In [ ]:
#Pearson's correlation coefficient

Correlation= df['Transect1'].corr(df['Combined Number of Individuals1'])

print(Correlation)



In [ ]:
#Chi- Square Test
import pandas as pd
from scipy import stats

crosstab = pd.crosstab(df['Transect1'], df['Combined Number of Individuals1'])
crosstab

stats.chi2_contingency(crosstab)

In [ ]:
#Graphing

import matplotlib.pyplot as plt

df.plot(x='Transect1', y='Combined Number of Individuals1', style='o')


In [ ]:
#FUTURE: Merge Rows with same column name to fill null values

#Masterdf.groupby(['Date\n(mm-dd-yyyy)']).first().reset_index()
